<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/search-word-wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [27]:
# 必要なライブラリをインストール
!pip install tqdm

import os
import json
from tqdm import tqdm

# 検索対象ディレクトリと単語
base_directory = "/content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output"  # JSONファイルが格納されているディレクトリ
exact_title = "手錠"  # 完全一致で検索するタイトル

def search_exact_title_and_exit(base_dir, exact_title):
    """
    指定したディレクトリ内のJSONファイルを再帰的に処理し、
    タイトルが完全一致する記事を見つけた時点で終了する。

    Parameters:
    - base_dir: 親ディレクトリのパス
    - exact_title: 完全一致で検索するタイトル
    """
    # ディレクトリを探索
    for root, _, files in os.walk(base_dir):
        json_files = [f for f in files if f.endswith(".json")]
        if not json_files:
            continue

        for json_file in tqdm(json_files, desc=f"処理中: {root}"):
            file_path = os.path.join(root, json_file)

            try:
                # JSONファイルを1行ずつ読み込み
                with open(file_path, 'r', encoding='utf-8') as infile:
                    for line in infile:
                        if not line.strip():  # 空行をスキップ
                            continue
                        try:
                            data = json.loads(line)
                            title = data.get("title", "")
                            text = data.get("text", "")

                            # タイトルが完全一致するかチェック
                            if title == exact_title:
                                print(f"\n【Title】: {title}")
                                print(f"【File Path】: {file_path}")
                                print(f"【Text】: \n{text}\n")
                                print("完全一致するタイトルが見つかりました。処理を終了します。")
                                return  # 見つかったら終了
                        except json.JSONDecodeError:
                            continue
            except Exception as e:
                print(f"エラー: {file_path} - {e}")

    print(f"\n検索完了: タイトルが「{exact_title}」と完全一致する記事は見つかりませんでした。")

# 関数の実行
search_exact_title_and_exit(base_directory, exact_title)


処理中: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AA: 100%|██████████| 100/100 [00:00<00:00, 114.98it/s]
処理中: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AB: 100%|██████████| 100/100 [00:00<00:00, 111.26it/s]
処理中: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AC: 100%|██████████| 100/100 [00:00<00:00, 116.50it/s]
処理中: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AD: 100%|██████████| 100/100 [00:00<00:00, 114.86it/s]
処理中: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AE:  92%|█████████▏| 92/100 [00:00<00:00, 111.67it/s]


【Title】: 手錠
【File Path】: /content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/AE/wiki_92.json
【Text】: 
手錠（てじょう、）とは、手を自己または他人が物理的に動きにくくする器具である。一般的には司法警察職員が被疑者を逮捕、拘束する時、また、保護しようとした精神錯乱者・酩酊者(泥酔者を含む)が暴れるなどして自己または他人の生命、身体または財産に危害を及ぼす事態において危険な場合などに用いる。
性質.
手錠は被拘束者の手の自由を不完全に奪うことができる。ただし下半身にはあまり制約が加えられないため、身体的には逃走は可能であり、衣類のボタンの掛け外しなどは支障なく行える。ただし腕を大きく振れないため、走ることには制約がつきまとい、バランスを取るのは難しい。その上開錠や破壊をせずに外すことはほぼ不可能なため、逃走した場合の被拘束者を示す格好の目印になる。また、こうしたことから、手錠や江戸時代の手鎖は、行動や表現の自由を形として奪うものという性格が指摘できる。ただし、現代日本において、連行用の手錠には捕縄（腰縄）が付けられており、手錠をはめると同時に捕縄で両手を腰部にくくりつけられるという形態であるため、より行動が制限される。「警察24時」でもPRのため、逮捕時に被疑者に手錠をかける様子を必ず収録させるが、その際に捕縄も手錠とともに装着させられることがある。
同種の拘束具に、両手の親指にはめて腕の自由を制限する指錠、足首にはめて逃走を防止する足錠というものもある（歩幅が大きく広がらないよう鎖で制限されているため、早く歩くのも困難）。
手錠が普及するまでは捕縄術が教授されていた。
逮捕と手錠.
手錠に対し、しばしば「被疑者を逮捕する際、手錠をかけなければならない」「手錠は逮捕するための道具である」と認識されることもあるが、検察官などによる逮捕執行には手錠は用いられない事を見ても分かるように、誤った認識である。前述の通り、「警察24時」でもPRのために手錠や捕縄をかける場面を収録させる。
手錠をかける場合、可動部が手首の上に来るようにかけ、ダブルロックする（手錠はただかけただけではどこまでも締まり続けるので、ダブルロックしない状態で暴れると必要以上に締め付けられ、